# Comparison between original model and model with delay

In [1]:
using JuMP
using HiGHS
using Ipopt
using Juniper

include("utils.jl") # demand_generator_mat, printTable, plotData

plotData (generic function with 1 method)

### Initial Parametrers

In [2]:
# Objective function Costs
c_cin = 1.5        # weight of served clients
c_ser = 2      # weight of server cost  
c_L = 1.5
c_phi = 1

# # Objective function
fobj(S, dr, Cin, phi, Z, L) = c_ser*sum(S) - c_cin*sum(Z) +c_phi*sum(L)
# fobj_os(S, dr, Cin, phi, Z, L) = c_ser*S -c_cin*Z + c_L*L


# Parameters
N = 10            # numner of examples
horiz = 20        # total horizon

# bounds
YM = 10           # max buffer legnth before dropping calls
XM = 6            # max queue length 
phiM = 10          # max adimission to queue
serM = 8          # number of servers
tserM = 3         # max service time

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 0

struct initial_conditions
    X0
    Y0
    L0
    Z0
end
ic = initial_conditions(X0, Y0, L0, Z0)

struct bounds
    XM
    YM
    phiM
    serM
    tserM
end
bds = bounds(XM, YM, phiM, serM, tserM)

rnd_tser = true
df_input = df_input_generator(horiz, serM, tserM, rnd_tser)

createDemands = true
if createDemands
    d_mat = demand_generator_mat(1, horiz, 7, "uniform", 1)
    a_mat = demand_generator_mat(1, horiz, 1,"uniform",0.5)
else
    d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
    a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
    
end

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls
   

### Original model

In [3]:
c_blr0 = 140
c_ser0 = 1

include("cc_os_original.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, S, b_opt, J = cc_os_original(ic, bds, c_ser0, c_blr0, d, a);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Problem is infeasible...
ok

In [ ]:
include("cc_om_original.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, S, b_opt, J = cc_om_original(ic, bds, c_ser0, c_blr0, d, a);

NUMERICAL_ERROR


In [9]:
include("cc_os_delay.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b1_opt, J = cc_os_delay(ic, bds, a, d, df_input, fobj);


In [ ]:
include("cc_om_delay.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b1_opt, J = cc_om_delay(ic, bds, a, d, df_input, fobj);

ok

(true, [4.0, 0.0, 0.0, 6.0, 3.0, 5.0, 5.0, 6.0, 6.0, 6.0  …  3.0, 4.0, 5.0, 6.0, 6.0, 6.0, 6.0, 5.0, 6.0, 6.0], [6.0, 8.0, 10.0, 7.0, 10.0, 10.0, 10.0, 10.0, 10.0, 9.0  …  10.0, 10.0, 10.0, 5.0, 9.0, 10.0, 8.0, 9.0, 10.0, 10.0], [0.0, 4.0, 7.0, 9.0, 12.0, 15.0, 18.0, 20.0, 24.0, 25.0  …  31.0, 32.0, 36.0, 40.0, 40.0, 44.0, 46.0, 47.0, 51.0, 55.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 2.0, 4.0, 5.0  …  5.0, 6.0, 6.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 10.0], [4.0, 5.0, 8.0, 3.0, 6.0, 4.0, 3.0, 4.0, 2.0, 1.0, 3.0, 3.0, 5.0, 6.0, 5.0, 5.0, 3.0, 2.0, 6.0, 4.0], [2.0, 5.0, 5.0, 3.0, 6.0, 4.0, 3.0, 4.0, 1.0, 1.0, 3.0, 3.0, 5.0, 1.0, 4.0, 5.0, 1.0, 1.0, 6.0, 4.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0], [0.0, 3.0, 8.0, 0.0, 6.0, 4.0, 3.0, 4.0, 2.0, 0.0, 3.0, 3.0, 5.0, 6.0, 0.0, 4.0, 3.0, 0.0, 5.0, 4.0], [4.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 4.0, 1.0, 2.0, 4.0, 1.0, 4.0, 4.0, 0.0, 4.0, 2.0, 1.0, 4.0, 4.0], [0.0, 0.0, 1.0, 5.0, 3.0, 2.0, 1.0

### Movel with delay

In [ ]:
# function optimize_model_delay(ic, bds, c_cin, c_ser, a, d, rnd_tser)
#     horiz = length(d)
     
#     bounds
#     XM = bds.XM
#     YM = bds.YM     
#     phiM = bds.phiM
#     serM = bds.serM

#     J = zeros(horiz)            # cost function
#     X = zeros(Int, horiz+1)     # current number of customers in queue x(k)S
#     Y = zeros(Int, horiz+1)     # current number of customers in buffer y(k)  
#     Z = zeros(Int, horiz+1)     # custumers served  
#     L = zeros(Int, horiz+1)     # number of customers lost

#     n = zeros(Int, horiz+1)     # number of empty slots in the queue
#     Q = zeros(Int, horiz+1)     # custumers entering queue
#     dr = zeros(Int, horiz+1)    # dropped due to full buffer

#     phi = zeros(Int, horiz+1)     # number of customers admitted to queue
#     Cin = zeros(Int, horiz+1)     # number of customers entering server
#     Cout = zeros(Int, horiz+1)    # number of customers leaving server

#     S = zeros(Int, horiz+1)                 # number of active servers
#     Sa = zeros(Bool, horiz+1, serM)         # server activation status (0 - active, 1 - inactive)
#     Sl = zeros(Int, horiz+1)                # number of free available servers
#     Sst = zeros(Bool, horiz+1, serM)        # server status (0 - free, 1 - busy)
#     Sc = zeros(Bool, horiz+1, serM, tserM)  # server conveyor
#     Sin = zeros(Bool, horiz+1, serM, tserM) # server input

#     Saux = zeros(Bool, horiz+1, serM)       # auxiliary server variable
#     b1_opt = zeros(horiz)
#     b2_opt = zeros(horiz)

#     B = zeros(Float64, horiz+1)             # balking rate

#     transition_matrix = zeros(Bool, tserM, tserM)
#     for i in 1:tserM-1
#         transition_matrix[i, i+1] = 1
#     end

#     if !rnd_tser
#         default_input = zeros(Bool, serM, tserM)  # default input (case with constant service time)
#         for i in 1:serM
#             default_input[i, 1] = 1
#         end
#     end

#     initial conditions
#     X[1] = ic.X0
#     Y[1] = ic.Y0
#     L[1] = ic.L0
#     Z[1] = ic.Z0

#     for t in 1:horiz
#         cc_lin_conv = Model(HiGHS.Optimizer)
#         set_silent(cc_lin_conv)

#         default_input = zeros(Bool, serM, tserM)  
#         for i in 1:serM
#             slot = rand(1:tserM)  # randomly select a slot for each server
#             default_input[i, slot] = 1
#         end

#         M1 = max(d[t], YM) + 10; # must be larger than d[t] and n[t] 
#         @variable(cc_lin_conv, b1, Bin);

#         M2 = max(X[t], serM) + 10; # must be larger than x[k] and Sl[k]
#         @variable(cc_lin_conv, b2, Bin);

#         @variable(cc_lin_conv, 0 <= XL[1:2] <= XM, Int)    
#         @variable(cc_lin_conv, 0 <= YL[1:2] <= YM, Int)     
#         @variable(cc_lin_conv, 0 <= ZL[1:2], Int)  
#         @variable(cc_lin_conv, 0 <= LL[1:2], Int)         
                            
#         @variable(cc_lin_conv, 0 <= nL <= YM, Int) 
#         @variable(cc_lin_conv, 0 <= QL, Int)      
#         @variable(cc_lin_conv, 0 <= drL, Int)   
        
#         @variable(cc_lin_conv, 0 <= phiL <= phiM, Int)   
#         @variable(cc_lin_conv, 0 <= CinL <= serM*mean_tser, Int)   
#         @variable(cc_lin_conv, 0 <= CoutL[1:2] <= serM, Int)    

#         @variable(cc_lin_conv, 0 <= SL <= serM, Int)  
#         @variable(cc_lin_conv, 0 <= SaL[1:serM], Bin)
#         @variable(cc_lin_conv, 0 <= SlL <= serM, Int)  
#         @variable(cc_lin_conv, 0 <= SstL[1:serM], Bin)  
#         @variable(cc_lin_conv, 0 <= ScL[1:2, 1:serM, 1:tserM], Bin) 
#         @variable(cc_lin_conv, 0 <= SinL[1:serM, 1:tserM], Bin)
#         @variable(cc_lin_conv, 0 <= SauxL[1:serM], Bin)

#         Initial conditions of buffer and queue for each optimization
#         @constraint(cc_lin_conv, XL[1] == X[t])
#         @constraint(cc_lin_conv, YL[1] == Y[t])
#         @constraint(cc_lin_conv, ZL[1] == Z[t])
#         @constraint(cc_lin_conv, LL[1] == L[t])
#         @constraint(cc_lin_conv, CoutL[1] == Cout[t])
#         @constraint(cc_lin_conv, ScL[1, :, :] == Sc[t, :, :])  # server conveyor status

#         Problem constraints  
#         @constraint(cc_lin_conv, XL[2] == XL[1] + phiL - a[t] - CinL)
#         @constraint(cc_lin_conv, YL[2] == YL[1] + QL - phiL)
#         @constraint(cc_lin_conv, ZL[2] == ZL[1] + CoutL[1])
#         @constraint(cc_lin_conv, LL[2] == LL[1] + drL + a[t])

#         @constraint(cc_lin_conv, nL == YM - YL[1] + phiL)  # number of empty slots in the queue
#         @constraint(cc_lin_conv, QL <= d[t])
#         @constraint(cc_lin_conv, QL <= nL)
#         @constraint(cc_lin_conv, QL >= d[t]-M1*b1)
#         @constraint(cc_lin_conv, QL >= nL-(1-b1)*M1)  

#         @constraint(cc_lin_conv, drL >= d[t]-nL)         
#         @constraint(cc_lin_conv, drL <= d[t]-QL) 

#         @constraint(cc_lin_conv, SL == serM - sum(SaL))              
#         @constraint(cc_lin_conv, SstL == sum(ScL[1,:,:], dims=2))  
#         @constraint(cc_lin_conv, SlL == SL - sum(SstL))    
#         @constraint(cc_lin_conv, [i=1:serM, j=1:tserM], SinL[i, j] == default_input[i, j] .* SauxL[i])
#         @constraint(cc_lin_conv, ScL[2, :, :] == ScL[1, :, :]*transition_matrix + SinL)

#         @constraint(cc_lin_conv, CinL <= XL[1]-a[t])
#         @constraint(cc_lin_conv, CinL <= SlL*mean_tser)
#         @constraint(cc_lin_conv, CinL >= XL[1]-a[t]-M2*b2)
#         @constraint(cc_lin_conv, CinL >= SlL*mean_tser-(1-b2)*M2)  
#         @constraint(cc_lin_conv, CoutL[2] == sum(ScL[1,:,tserM]))  

#         @constraint(cc_lin_conv, SauxL <= ones(Bool, serM) - SstL - SaL)  
#         @constraint(cc_lin_conv, CinL <= mean_tser*sum(SauxL))  
#         @constraint(cc_lin_conv, CinL >= mean_tser*sum(SauxL)-mean_tser+1)  

#         Objective function
#         @objective(cc_lin_conv, Min, LL[2] + c_ser*SL - c_cin*CinL - phiL) # design better objective function

#         JuMP.optimize!(cc_lin_conv)
        
#         status = termination_status(cc_lin_conv)
#         if (status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED || status==MOI.ALMOST_LOCALLY_SOLVED) && has_values(cc_lin_conv)
#             Save computed values
#             X[t+1] = JuMP.value(XL[2]);
#             Y[t+1] = JuMP.value(YL[2]);
#             Z[t+1] = JuMP.value(ZL[2]);
#             L[t+1] = JuMP.value(LL[2]);

#             n[t] = JuMP.value(nL);
#             Q[t] = JuMP.value(QL);  
#             dr[t] = JuMP.value(drL);
            
#             phi[t] = JuMP.value(phiL);
#             Cin[t] = JuMP.value(CinL);
#             Cout[t+1] = JuMP.value(CoutL[2]);

#             S[t] = JuMP.value(SL);
#             Sl[t] = JuMP.value(SlL);
#             Sst[t, :] = JuMP.value.(SstL);
#             Sc[t+1, :, :] = JuMP.value.(ScL[2, :, :]);
#             Sin[t, :, :] = JuMP.value.(SinL);
#             Saux[t, :] = JuMP.value.(SauxL);

#             b1_opt[t] = JuMP.value(b1);
#             b2_opt[t] = JuMP.value(b2);

#             J[t] = objective_value(cc_lin_conv) # Repeated line
            
#             optimal = true;
#             Display results
#             println("Optimal solution:")
#             println("Objective value = ", objective_value(cc_lin_conv))
#         else
#             optimal = false;
#             println(t, " ", status)
#             break
#         end
#     end

#     return optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sst, Sc, Sin, Saux, b1_opt, b2_opt, J

# end

# function optimize_model_original(ic, bds, c_blr, c_ser, a, d)
#     horiz = length(d)

#     # Check for optimality
#     optimal=false

#     # Output variables
#     J = zeros(horiz)     #Cost function
#     X  = zeros(horiz+1)  #Server queue length, XM is max server queue length
#     Y = zeros(horiz+1)   #Buffer queue length, YM is max buffer capacity
#     ser = zeros(horiz)   #Number of servers, ser
#     phi = zeros(horiz)   #Transfer rate from buffer to server queue
#     dr = zeros(horiz)    #Dropping rate of new arrivals to buffer
#     n  = zeros(horiz)    #Number of empty spots in buffer queue
#     Q  = zeros(horiz)    #Transfer rate from demand into input buffer
#     L  = zeros(horiz+1)  #Lost (dropped)
#     Z  = zeros(horiz+1)  #Served
#     b_opt = zeros(horiz) #Binary variable for big-M logic to algebra trick
#     blr = zeros(horiz+1)


#     # Initial values of display variables
#     X[1] = ic.X0
#     Y[1] = ic.Y0
#     L[1] = ic.L0
#     Z[1] = ic.Z0
#     blr[1] = L[1]/(L[1] + Z[1])
    
#     # Optimization loop
#     for t in 1:horiz  
 
#         # Setting up solvers and their attributes
#         # Optimization problem
#         ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 1)
#         highs = optimizer_with_attributes(HiGHS.Optimizer, "output_flag" => false)
#         cc_nl_fobj_os = Model(
#             optimizer_with_attributes(
#                 Juniper.Optimizer, 
#                 "nl_solver" => ipopt,
#                 "mip_solver" => highs, 
#                 "allow_almost_solved" => false,
#                 "feasibility_pump" => true        
#                 )
#         )        
        
#         # No screen output
#         set_silent(cc_nl_fobj_os)
        
#         # For the integer constraint on variable b
#         M = max(d[t], bds.YM)+10; # must be larger than d[t] and n[t]    
        
#         # Optimization variables
#         @variable(cc_nl_fobj_os, 0 <= XL[1:2] <= bds.XM, Int)
#         @variable(cc_nl_fobj_os, 0 <= YL[1:2] <= bds.YM, Int)
#         @variable(cc_nl_fobj_os, 0 <= serL <= bds.serM, Int)
#         @variable(cc_nl_fobj_os, 0 <= phiL <= bds.phiM, Int)
#         @variable(cc_nl_fobj_os, 0 <= nL <= bds.YM, Int)
#         @variable(cc_nl_fobj_os, 0 <= drL, Int)
#         @variable(cc_nl_fobj_os, 0 <= QL, Int)
#         @variable(cc_nl_fobj_os, b , Bin);
#         @variable(cc_nl_fobj_os, 0<=ZL[1:2], Int)
#         @variable(cc_nl_fobj_os, 0<=LL[1:2], Int)

#         # Initial conditions of buffer and queue for each optimization
#         @constraint(cc_nl_fobj_os, XL[1] == X[t])
#         @constraint(cc_nl_fobj_os, YL[1] == Y[t])
#         @constraint(cc_nl_fobj_os, LL[1] == L[t])
#         @constraint(cc_nl_fobj_os, ZL[1] == Z[t])

#         # Problem constraints    
#         @constraint(cc_nl_fobj_os, XL[2] == XL[1] + phiL - serL - a[t] )
#         @constraint(cc_nl_fobj_os, YL[2] == YL[1] + QL - phiL )
#         @constraint(cc_nl_fobj_os, nL   == bds.YM + phiL - YL[1])      
#         @constraint(cc_nl_fobj_os, drL >= d[t]-nL)         
#         @constraint(cc_nl_fobj_os, drL <= d[t]-QL) 

#         @constraint(cc_nl_fobj_os, QL <= d[t])
#         @constraint(cc_nl_fobj_os, QL <= nL)
#         @constraint(cc_nl_fobj_os, QL >= d[t]-M*b )
#         @constraint(cc_nl_fobj_os, QL >= nL-(1-b)*M)    

#         @constraint(cc_nl_fobj_os, LL[2] == LL[1] + drL + a[t])
#         @constraint(cc_nl_fobj_os, ZL[2] == ZL[1] + serL)         

#         @constraint(cc_nl_fobj_os, 0 <= serL <= bds.serM)
#         #@constraint(CallCenter_L_osa, serM - 2 <= serL <= serM)
#         @constraint(cc_nl_fobj_os, 0 <= phiL <= bds.phiM)
#         @constraint(cc_nl_fobj_os, 0 <= XL[2] <= bds.XM)
#         @constraint(cc_nl_fobj_os, 0 <= YL[2] <= bds.YM)
       
#         # Objective function        
#         @NLobjective(cc_nl_fobj_os, Min, c_ser*serL + c_blr*LL[2]/(LL[2] + ZL[2]) )
    
#         # Compute solution    
#         JuMP.optimize!(cc_nl_fobj_os)

#         # Print the model
#         #print(cc_nl_fobj_os)
   
#         # Save objective value
#         J[t] = objective_value(cc_nl_fobj_os) #Repeated line
        
#         # Check if solver found optimal solution
#         status = termination_status(cc_nl_fobj_os)
#         if (status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED || status==MOI.ALMOST_LOCALLY_SOLVED) && has_values(cc_nl_fobj_os)
#             # Get and save solution of each optimization problem
#             X[t+1] = JuMP.value(XL[2]);
#             Y[t+1] = JuMP.value(YL[2]);
#             L[t+1] = JuMP.value(LL[2]);
#             Z[t+1] = JuMP.value(ZL[2]);
#             blr[t+1] = L[t+1]/(L[t+1] + Z[t+1])
#             dr[t]  = JuMP.value(drL);
#             n[t]   = JuMP.value(nL);
#             ser[t] = JuMP.value(serL);
#             phi[t] = JuMP.value(phiL);
#             Q[t]   = JuMP.value(QL);  
#             b_opt[t] = JuMP.value(b);

#             J[t] =  objective_value(cc_nl_fobj_os)

#             # Informs the calling function an optimal solution was found
#             optimal = true;      
#             # println("Optimal solution:")
#             # println("Objective value = ", objective_value(cc_nl_fobj_os))         
#         else
#             println("Problem is infeasible...")
            
#             # X = zeros(horiz+1)
#             # Y = zeros(horiz+1)
#             # ser = zeros(horiz)
#             # blr = zeros(horiz+1)
    
#             # Informs the calling function the problem is infeasible for this demand
#             optimal = false;
#             println(t, " ", status)
#             break
#         end
#     end

#     return optimal, X, Y, Z, L, n, Q, dr, phi, ser, b_opt, J
# end 

optimize_model_delay (generic function with 1 method)